# Lab 5

## Context Engineering

We want to develop several types of Memory:

1. Long Term Memory - graph

A knowledge graph as a persistent store of entities

2. Long Term Memory - knowledge

A RAG database of Q&A and any other useful information

3. Permanent context

Summary and linkedin profile included in everything

4. FAQ

A fixed list of 

In [28]:
from dotenv import load_dotenv
from agents.mcp import MCPServerStdio
import sqlite3
load_dotenv(override=True)

True

In [29]:
memory_graph_params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./twin/memory/graph.db"}}

In [30]:
async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as memory_graph:
    memory_graph_tools = await memory_graph.session.list_tools()

memory_graph_tools.tools

[Tool(name='create_entities', title=None, description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, outputSchema=None, annotations=None, meta=None),
 Tool(name='search_nodes', title=None, description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, outputS

In [31]:
memory_rag_params = {
    "command": "uvx",
    "args": ["mcp-server-qdrant"],
    "env": {
        "QDRANT_LOCAL_PATH": "./twin/memory/knowledge/",
        "COLLECTION_NAME": "knowledge",
        "EMBEDDING_MODEL": "sentence-transformers/all-MiniLM-L6-v2"
    }
}

In [32]:
async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as memory_rag:
    memory_rag_tools = await memory_rag.session.list_tools()

memory_rag_tools.tools

[Tool(name='qdrant-find', title=None, description='Look up memories in Qdrant. Use this tool when you need to: \n - Find memories by their content \n - Access memories for further analysis \n - Get some personal information about the user', inputSchema={'properties': {'query': {'description': 'What to search for', 'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}, outputSchema={'properties': {'result': {'items': {'type': 'string'}, 'title': 'Result', 'type': 'array'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None),
 Tool(name='qdrant-store', title=None, description='Keep the memory for later use, when you are asked to remember something.', inputSchema={'properties': {'information': {'description': 'Text to store', 'title': 'Information', 'type': 'string'}, 'metadata': {'anyOf': [{'additionalProperties': True, 'type': 'object'}, {'type': 'null'}], 'default': None, 'descriptio

In [2]:
DB = "./twin/memory/questions.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS questions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT,
            answer TEXT
        )
    ''')
    conn.commit()

In [25]:
def record_question_with_no_answer(question: str) -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO questions (question, answer) VALUES (?, NULL)", (question,))
        conn.commit()
        return "Recorded question with no answer"

def get_questions_with_no_answer() -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id, question FROM questions WHERE answer IS NULL")
        rows = cursor.fetchall()
        if rows:
            return "\n".join(f"Question id {row[0]}: {row[1]}" for row in rows)
        else:
            return "No questions with no answer found"

def get_questions_with_answer() -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT question, answer FROM questions WHERE answer IS NOT NULL")
        rows = cursor.fetchall()
        return "\n".join(f"Question: {row[0]}\nAnswer: {row[1]}\n" for row in rows)

def record_answer_to_question(id: int, answer: str) -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("UPDATE questions SET answer = ? WHERE id = ?", (answer, id))
        conn.commit()
        return "Recorded answer to question"

In [26]:
get_questions_with_no_answer()

'No questions with no answer found'

In [27]:
print(get_questions_with_answer())

Question: What is your favorite color?
Answer: Orange

Question: What is your favorite fruit?
Answer: Banana



In [17]:
record_question_with_no_answer("What is your favorite fruit?")

In [19]:
record_answer_to_question(2, "Banana")